In [75]:
import pandas as pd
import os
import glob
from shutil import copyfile
import datetime
from pathlib import Path
import win32com.client
import time
import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
import schedule
import time


In [4]:
os.chdir(r'C:\Users\Zoidb\OneDrive\Documents\Automation')

In [6]:
df = pd.read_excel(r'C:\Users\Zoidb\OneDrive\Documents\Automation\Financial Sample.xlsx')

In [8]:
file_name = r'C:\Users\Zoidb\OneDrive\Documents\Automation\newest_raw_data.csv'
df.to_csv(file_name, sep = '|', index = False)

In [57]:
##Finding most recently uploaded file from a folder

list_of_reports = glob.glob(r'C:\Users\Zoidb\OneDrive\Documents\Automation\Weekly Reports\*')
latest_file = max(list_of_reports, key = os.path.getctime)
latest_file = os.path.basename(latest_file)

latest_file

'Financial Sample Week 3.xlsx'

In [66]:
## Creating a copy of the latest file and saving it to folder

today_date = datetime.datetime.today().strftime('%Y%m%d')
src = Path('C:/Users/Zoidb/OneDrive/Documents/Automation/Weekly Reports') / latest_file
new_file = today_date + '_latest_periodic_report.xlsx'
dst = Path('C:/Users/Zoidb/OneDrive/Documents/Automation/Weekly Reports') / new_file
copyfile(src,dst)


WindowsPath('C:/Users/Zoidb/OneDrive/Documents/Automation/Weekly Reports/20200928_latest_periodic_report.xlsx')

In [72]:
## Opening the new file in excel and refreshing when a new latest file is created

SourcePathName = Path('C:/Users/Zoidb/OneDrive/Documents/Automation/Weekly Reports') / new_file
Application = win32com.client.DispatchEx('Excel.Application')
Application.DisplayAlerts = False
Application.Visible = 1
Workbook = Application.Workbooks.Open(SourcePathName)
Workbook.RefreshAll()
time.sleep(20)
Workbook.Save()
Workbook.Close()
Application.Quit()

In [70]:
##Automating an email being sent went the latest report is created

fromaddr = 'johndoe@company.com'
toaddr = 'janedoe@company.com'
body = 'The latest analysis report has been updated and saved in ABC location'

msg = MIMEMultipart()
msg['From'] = fromaddr
msg['To'] = toaddr
msg['Subject'] = 'Periodic Analysis Report'
msg.attach(MIMEText(body, 'plain'))

server = smtplib.SMTP('smtp-name', 'port-number-without-quotes')
server.starttls()
server.login(fromaddr, 'password')
text = msg.as_string()
server.sendmail(fromaddr, toaddr, text)
server.quit()


WindowsPath('C:/Users/Zoidb/OneDrive/Documents/Automation/Weekly Reports/20200928_latest_periodic_report.xlsx')

In [ ]:
## Scheduling this code to run at a certain time, Monday at 9:00am for example

In [77]:
def generate_weekly_report():
    list_of_reports = glob.glob(r'C:\Users\Zoidb\OneDrive\Documents\Automation\Weekly Reports\*')
    latest_file = max(list_of_reports, key = os.path.getctime)
    latest_file = os.path.basename(latest_file)
    today_date = datetime.datetime.today().strftime('%Y%m%d')
    src = Path('C:/Users/Zoidb/OneDrive/Documents/Automation/Weekly Reports') / latest_file
    new_file = today_date + '_latest_periodic_report.xlsx'
    dst = Path('C:/Users/Zoidb/OneDrive/Documents/Automation/Weekly Reports') / new_file
    copyfile(src,dst)
    SourcePathName = Path('C:/Users/Zoidb/OneDrive/Documents/Automation/Weekly Reports') / new_file
    Application = win32com.client.DispatchEx('Excel.Application')
    Application.DisplayAlerts = False
    Application.Visible = 1
    Workbook = Application.Workbooks.Open(SourcePathName)
    Workbook.RefreshAll()
    time.sleep(20)
    Workbook.Save()
    Workbook.Close()
    Application.Quit()
    

schedule.every().monday.at('11:00').do(generate_weekly_report)

while True:
    schedule.run_pending()
    time.sleep(1)

KeyboardInterrupt: 